In [127]:
import pandas as pd
import numpy as np

from sklearn.metrics import fbeta_score

pd.options.display.max_colwidth=100
pd.options.display.max_columns=300

employees = pd.read_csv("../data/employees.csv")
history = pd.read_csv("../data/history.csv")
submission = pd.read_csv("../data/submission.csv")

# history.loc[:,'Date'] = list(pd.to_datetime(history['Date']))

def get_month(text):
    if type(text) == str:
        numbers = text.split("/")
        return int(numbers[0])

def get_year(text):
    if type(text) == str:
        numbers = text.split("/")
        return int(numbers[-1])

df = history.merge(employees)

# for data labeling
def label_df(data, n_month=3):
    labels = []
    for emp in data.EmployeeID.unique():
        curr_emp = list(data[data.EmployeeID == emp]['DismissalDate'])
        len_emp = len(curr_emp)
        if pd.isnull(curr_emp[0]):
            labels += [0 for _ in range(len_emp - n_month)] + [2 for _ in range(n_month)]
        else:
            labels += [0 for _ in range(len_emp - n_month)] + [1 for _ in range(n_month)]
    return labels

lbls = label_df(df)

In [128]:
df['target'] = lbls
df = df[df.target!=2]

In [129]:
columns_to_drop = ['EmployeeID', 'HiringDate', 'DismissalDate']

cat_columns = ['DevCenterID', 'SBUID', 'PositionID', 'PositionLevel', 
               'IsTrainee', 'LanguageLevelID', 'CustomerID', 'ProjectID', 
               'IsInternalProject', 'OnSite', 'CompetenceGroupID', 'FunctionalOfficeID',
               'PaymentTypeId']

X = df.drop(columns_to_drop, axis = 1)

In [130]:
from category_encoders.basen import BaseNEncoder
encoder = BaseNEncoder(cols = cat_columns, base = 2)
X = encoder.fit_transform(X)

In [131]:
# !pip install imblearn
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
import random

from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras import backend as K
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# Visualize training history
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.regularizers import l2
import matplotlib.pyplot as plt
import numpy
import keras

from keras import backend as K

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    
    beta = 1.7
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    
    return (1+beta**2)

# model.compile(loss='binary_crossentropy',
#           optimizer= "adam",
#           metrics=[f1])

RANDOM_SEED = 42

def set_seed(random_state=RANDOM_SEED):
    random.seed(random_state)
    np.random.seed(random_state)

set_seed()

def model_fit(X_train, y_train):
    clf2 = BalancedRandomForestClassifier(n_estimators=200, n_jobs = 8)
    clf2 = clf2.fit(X_train, y_train)
    return clf2

def model_predict(X, models):
    preds = models.predict(X)
    return preds




# def model_fit(X_train, y_train):
#     # create model
#     model = Sequential()
#     model.add(Dense(1500, input_dim=X_train.shape[1], activation='relu', activity_regularizer=l2(0.00001)))
#     model.add(Dense(500, activation='relu', activity_regularizer=l2(0.00001)))
#     model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
    
#     # Compile model
#     optimizer = keras.optimizers.Adam(lr=0.001)
#     model.compile(loss='binary_crossentropy', optimizer=optimizer)
#     history = model.fit(X_train, y_train, validation_split=0.1, epochs=5, batch_size=512, verbose=1)
#     return model

# def model_predict(X, models):
#     preds = models.predict(X)
#     return preds

In [132]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

USE_SCALER = True
RANDOM_SEED = 42

X["month"] = X["Date"].apply(get_month)
X["year"] = X["Date"].apply(get_year)
X = X.sort_values(["year","month"])
X["year_month"] = X.apply(lambda row: str(row["year"])+"_"+str(row["month"]), axis=1)
year_month = list(X.year_month.unique())
mapping_year_month = dict(zip(year_month, range(len(year_month))))
X["order_in_time"] = X["year_month"].map(mapping_year_month)
splits = [[[0,9],[10,11]],
          [[0,11],[12,13]],
          [[0,13],[14,15]],
          [[0,15],[16,17]],
          [[0,17],[18,19]]]

fold = 0
scores = []

kf = KFold(n_splits=5, random_state=RANDOM_SEED, shuffle=True)
kf.get_n_splits(X)

# for split in splits:
#     fold += 1
    
#     X_train = X.query(f"order_in_time >= "+str(split[0][0])+" & order_in_time <"+str(split[0][1]))
#     X_val = X.query(f"order_in_time >= "+str(split[1][0])+" & order_in_time <"+str(split[1][1]))
    
#     y_train = X_train['target']
#     y_val = list(X_val['target'])
    
#     X_train =X_train.drop(columns=["month", "year", "year_month", "order_in_time", "Date",'target'])
#     X_val =X_val.drop(columns=["month", "year", "year_month", "order_in_time", "Date",'target'])

tmp = 0
tmp2 = 0

y = X.target
X = X.drop(columns=["month", "year", "year_month", "order_in_time", "Date",'target'])

for train_index, test_index in kf.split(X):
    fold += 1
    X_train, X_val = X.loc[X.index.intersection(train_index)], X.loc[X.index.intersection(test_index)]
    y_train, y_val = y.loc[y.index.intersection(train_index)], y.loc[y.index.intersection(test_index)]
    
    if USE_SCALER:
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)
    
    print("fold: ", fold)
    print("train:", len(X_train))
    print("val:",len(X_val))
    print(" ")
    
    print('FOLD #{}'.format(fold))
    models = model_fit(X_train, y_train)
    print('END OF MODEL FIT')
    
    y_pred = model_predict(X_val, models)
    
    tmp = y_pred
    tmp2 = y_val

    score = fbeta_score(y_val, y_pred, beta=1.7)
    
    print('Validation Score: {}'.format(score))
    scores.append(score)
    
mean_score = np.mean(scores)
print('MEAN OF SCOREs: {}'.format(mean_score))   

model = model_fit(X, y)   

fold:  1
train: 50879
val: 12697
 
FOLD #1
END OF MODEL FIT
Validation Score: 0.42974819211834264
fold:  2
train: 50859
val: 12717
 
FOLD #2
END OF MODEL FIT
Validation Score: 0.4164808435567681
fold:  3
train: 50893
val: 12683
 
FOLD #3
END OF MODEL FIT
Validation Score: 0.4302827517187582
fold:  4
train: 50809
val: 12767
 
FOLD #4
END OF MODEL FIT
Validation Score: 0.42244825115876394
fold:  5
train: 50864
val: 12712
 
FOLD #5
END OF MODEL FIT
Validation Score: 0.4194693352197921
MEAN OF SCOREs: 0.423685874754485


In [138]:
from datetime import datetime

history.loc[:,'Date'] = list(pd.to_datetime(history['Date']))

df = history.merge(employees)
df['target'] = lbls

X_test = df[df.target==2]
X_test = X_test[X_test.Date == datetime(2019,2,1)]

X_test = X_test[X_test.EmployeeID.isin(set(submission.EmployeeID))]

In [139]:
emp_ids = X_test.EmployeeID
X_test = X_test.drop(columns_to_drop, axis = 1)

X_test = encoder.transform(X_test)

X_test = X_test.drop(columns=["Date",'target'])

In [140]:
preds = model.predict(X_test)

In [143]:
result = pd.DataFrame({'EmployeeID':emp_ids, 'target':preds})

In [144]:
result.to_csv('Mykola_initial_submit.csv', index = False)

In [145]:
submission = pd.read_csv("../data/submission.csv")
len(submission.EmployeeID.unique())

4156